# Diagnostic 14: Corrected Strategy Validation

**Purpose:** Validate storage cost handling in corrected prediction strategies

**Key Tests:**
1. Compare buggy vs fixed vs CORRECTED strategies with synthetic_acc90
2. Prove corrected logic works (net_benefit as PRIMARY driver)
3. Demonstrate parameter sensitivity returns

**Expected Results:**
- Buggy: ~$708k - sold when should hold
- Fixed: ~$223k - held indefinitely, 9% storage costs!
- CORRECTED: Beat baseline - rational net_benefit decisions

**The Fix:**
```python
if net_benefit > threshold:
    HOLD  # Profitable after storage costs
elif net_benefit < 0:
    SELL  # Storage costs exceed gains
```

In [0]:
%run ../00_setup_and_config

In [0]:
import sys
import os
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta
import importlib.util

print("="*80)
print("DIAGNOSTIC 14: CORRECTED STRATEGY VALIDATION")
print("="*80)
print("\nGoal: Prove corrected storage cost logic beats baseline with 90% accuracy")

## Load All Strategy Versions

In [0]:
# Load FIXED strategies (flawed - defer logic)
spec = importlib.util.spec_from_file_location("fixed_strategies", "fixed_strategies.py")
fixed_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fixed_module)

ExpectedValueStrategyFixed = fixed_module.ExpectedValueStrategyFixed
ConsensusStrategyFixed = fixed_module.ConsensusStrategyFixed
RiskAdjustedStrategyFixed = fixed_module.RiskAdjustedStrategyFixed

print("✓ Loaded FIXED strategies (defer bug)")

# Load CORRECTED strategies (net_benefit driven)
spec = importlib.util.spec_from_file_location("corrected_strategies", "corrected_strategies.py")
corrected_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(corrected_module)

ExpectedValueStrategyCorrected = corrected_module.ExpectedValueStrategyCorrected
ConsensusStrategyCorrected = corrected_module.ConsensusStrategyCorrected
RiskAdjustedStrategyCorrected = corrected_module.RiskAdjustedStrategyCorrected

print("✓ Loaded CORRECTED strategies (net_benefit primary driver)")

## Load Data

In [0]:
COMMODITY = 'coffee'
MODEL_VERSION = 'synthetic_acc90'

DATA_PATHS = get_data_paths(COMMODITY, MODEL_VERSION)
COMMODITY_CONFIG = COMMODITY_CONFIGS[COMMODITY]

print("Loading prices...")
prices_table = get_data_paths(COMMODITY)['prices_prepared']
prices = spark.table(prices_table).toPandas()
prices['date'] = pd.to_datetime(prices['date'])
print(f"✓ Loaded {len(prices)} price records")

print("\nLoading predictions...")
matrices_path = DATA_PATHS['prediction_matrices']
with open(matrices_path, 'rb') as f:
    prediction_matrices = pickle.load(f)
print(f"✓ Loaded {len(prediction_matrices)} prediction matrices")

# Normalize dates
prediction_matrices = {pd.to_datetime(k): v for k, v in prediction_matrices.items()}

print("\nLoading baseline results for comparison...")
results_path = DATA_PATHS['results_detailed']
with open(results_path, 'rb') as f:
    baseline_results = pickle.load(f)
print(f"✓ Loaded baseline results")

## Create Diagnostic Backtest Engine

In [0]:
class DiagnosticBacktestEngine:
    """
    Simplified backtest engine for diagnostic testing.
    """
    
    def __init__(self, prices_df, prediction_matrices, commodity_config):
        self.prices = prices_df
        self.prediction_matrices = prediction_matrices
        self.config = commodity_config
        
    def run_backtest(self, strategy, initial_inventory=50.0):
        """Run backtest for a single strategy"""
        
        inventory = initial_inventory
        trades = []
        total_revenue = 0
        total_transaction_costs = 0
        total_storage_costs = 0
        
        strategy.reset()
        strategy.set_harvest_start(0)
        
        for day in range(len(self.prices)):
            current_date = self.prices.iloc[day]['date']
            current_price = self.prices.iloc[day]['price']
            
            price_history = self.prices.iloc[:day+1].copy()
            predictions = self.prediction_matrices.get(current_date, None)
            
            decision = strategy.decide(
                day=day,
                inventory=inventory,
                current_price=current_price,
                price_history=price_history,
                predictions=predictions
            )
            
            if decision['action'] == 'SELL' and decision['amount'] > 0:
                amount = min(decision['amount'], inventory)
                
                price_per_ton = current_price * 20
                revenue = amount * price_per_ton
                transaction_cost = revenue * (self.config['transaction_cost_pct'] / 100)
                
                total_revenue += revenue
                total_transaction_costs += transaction_cost
                inventory -= amount
                
                trades.append({
                    'day': day,
                    'date': current_date,
                    'amount': amount,
                    'price': current_price,
                    'revenue': revenue,
                    'transaction_cost': transaction_cost,
                    'reason': decision['reason']
                })
            
            if inventory > 0:
                avg_price = self.prices.iloc[:day+1]['price'].mean()
                price_per_ton = avg_price * 20
                storage_cost = inventory * price_per_ton * (self.config['storage_cost_pct_per_day'] / 100)
                total_storage_costs += storage_cost
        
        net_earnings = total_revenue - total_transaction_costs - total_storage_costs
        
        return {
            'strategy': strategy.name,
            'net_earnings': net_earnings,
            'total_revenue': total_revenue,
            'total_transaction_costs': total_transaction_costs,
            'total_storage_costs': total_storage_costs,
            'num_trades': len(trades),
            'final_inventory': inventory,
            'trades': trades
        }

print("✓ Diagnostic backtest engine created")

## Test: Buggy vs Fixed vs Corrected

In [0]:
print("="*80)
print("BUGGY vs FIXED vs CORRECTED COMPARISON")
print("="*80)

engine = DiagnosticBacktestEngine(prices, prediction_matrices, COMMODITY_CONFIG)

# Get baseline and buggy results
best_baseline_name = 'Equal Batches'
best_baseline_earnings = baseline_results[best_baseline_name]['net_earnings']
buggy_ev_earnings = baseline_results['Expected Value']['net_earnings']

print(f"\nBest Baseline ({best_baseline_name}): ${best_baseline_earnings:,.2f}")
print(f"Buggy Expected Value: ${buggy_ev_earnings:,.2f} ({100*(buggy_ev_earnings - best_baseline_earnings)/best_baseline_earnings:+.2f}%)")

# Test FIXED strategy (should lose badly due to storage costs)
print("\n" + "="*80)
print("FIXED Strategy (defer bug fix - but storage cost flaw!)")
print("="*80)

fixed_ev = ExpectedValueStrategyFixed(
    storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
    transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct'],
    min_ev_improvement=50,
    baseline_batch=0.15,
    baseline_frequency=10
)

fixed_result = engine.run_backtest(fixed_ev)
fixed_vs_baseline = 100 * (fixed_result['net_earnings'] - best_baseline_earnings) / best_baseline_earnings

print(f"\nFixed Expected Value: ${fixed_result['net_earnings']:,.2f} ({fixed_vs_baseline:+.2f}% vs baseline)")
print(f"  Trades: {fixed_result['num_trades']}")
print(f"  Storage costs: ${fixed_result['total_storage_costs']:,.2f}")
print(f"  Final inventory: {fixed_result['final_inventory']:.1f} tons")

if fixed_vs_baseline < -50:
    print(f"\n⚠️  CONFIRMED: Fixed strategy loses {abs(fixed_vs_baseline):.1f}% due to excessive storage costs!")

# Test CORRECTED strategy (should beat baseline)
print("\n" + "="*80)
print("CORRECTED Strategy (net_benefit driven)")
print("="*80)

corrected_ev = ExpectedValueStrategyCorrected(
    storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
    transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct'],
    min_net_benefit=50,
    negative_threshold=-20,
    high_confidence_cv=0.05,
    medium_confidence_cv=0.10,
    strong_trend_adx=25,
    batch_positive_confident=0.0,
    batch_positive_uncertain=0.05,
    batch_marginal=0.15,
    batch_negative_mild=0.20,
    batch_negative_strong=0.30,
    cooldown_days=7,
    baseline_batch=0.15,
    baseline_frequency=30
)

corrected_result = engine.run_backtest(corrected_ev)
corrected_vs_baseline = 100 * (corrected_result['net_earnings'] - best_baseline_earnings) / best_baseline_earnings

print(f"\nCorrected Expected Value: ${corrected_result['net_earnings']:,.2f} ({corrected_vs_baseline:+.2f}% vs baseline)")
print(f"  Trades: {corrected_result['num_trades']}")
print(f"  Storage costs: ${corrected_result['total_storage_costs']:,.2f}")
print(f"  Final inventory: {corrected_result['final_inventory']:.1f} tons")

if corrected_vs_baseline > 0:
    print(f"\n✓✓✓ SUCCESS! Corrected strategy beats baseline by {corrected_vs_baseline:.1f}%!")
else:
    print(f"\n⚠️  Still losing - further investigation needed")

# Show the swing
print("\n" + "="*80)
print("COMPARISON SUMMARY")
print("="*80)

swing_fixed_to_corrected = corrected_vs_baseline - fixed_vs_baseline

print(f"\nBaseline: ${best_baseline_earnings:,.2f}")
print(f"Buggy:    ${buggy_ev_earnings:,.2f} ({100*(buggy_ev_earnings - best_baseline_earnings)/best_baseline_earnings:+.2f}%)")
print(f"Fixed:    ${fixed_result['net_earnings']:,.2f} ({fixed_vs_baseline:+.2f}%)")
print(f"Corrected: ${corrected_result['net_earnings']:,.2f} ({corrected_vs_baseline:+.2f}%)")

print(f"\nSwing (Fixed → Corrected): {swing_fixed_to_corrected:+.1f} percentage points")
print(f"Improvement: ${corrected_result['net_earnings'] - fixed_result['net_earnings']:,.2f}")

## Analyze Trade Reasons (Corrected)

In [0]:
print("="*80)
print("TRADE REASON ANALYSIS: CORRECTED STRATEGY")
print("="*80)

corrected_trades = corrected_result['trades']

# Count different reason types
reason_counts = {}
for trade in corrected_trades:
    reason = trade['reason']
    # Extract reason type (first word)
    reason_type = reason.split('_')[0]
    reason_counts[reason_type] = reason_counts.get(reason_type, 0) + 1

print(f"\nTotal trades: {len(corrected_trades)}")
print(f"\nTrade reasons:")
for reason_type, count in sorted(reason_counts.items(), key=lambda x: -x[1]):
    print(f"  {reason_type}: {count} trades")

# Sample some trades
print(f"\nSample trades:")
for i, trade in enumerate(corrected_trades[:10]):
    print(f"  {i+1}. Day {trade['day']}: {trade['amount']:.1f}t @ ${trade['price']:.2f}/lb - {trade['reason']}")

## Parameter Sensitivity Test

In [0]:
print("="*80)
print("PARAMETER SENSITIVITY TEST")
print("="*80)
print("\nTesting that different parameters → different results")

# Test min_net_benefit sensitivity
min_benefit_values = [30, 50, 75, 100]
min_benefit_results = []

print(f"\nTesting min_net_benefit = {min_benefit_values}")
for val in min_benefit_values:
    strategy = ExpectedValueStrategyCorrected(
        storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
        transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct'],
        min_net_benefit=val,
        negative_threshold=-20,
        batch_positive_confident=0.0,
        batch_negative_strong=0.30
    )
    result = engine.run_backtest(strategy)
    min_benefit_results.append(result['net_earnings'])
    print(f"  min_net_benefit={val}: ${result['net_earnings']:,.2f}")

# Check if we got variation
unique_results = len(set(min_benefit_results))
if unique_results > 1:
    print(f"\n✓ SENSITIVITY CONFIRMED: {unique_results} different results from {len(min_benefit_values)} parameter values")
    earnings_range = max(min_benefit_results) - min(min_benefit_results)
    print(f"  Range: ${earnings_range:,.2f}")
else:
    print(f"\n⚠️  NO SENSITIVITY: All parameters gave same result (${min_benefit_results[0]:,.2f})")

# Test batch sizing sensitivity
batch_configs = [
    {'batch_negative_strong': 0.20, 'batch_negative_mild': 0.15},
    {'batch_negative_strong': 0.30, 'batch_negative_mild': 0.20},
    {'batch_negative_strong': 0.40, 'batch_negative_mild': 0.25},
]

print(f"\nTesting batch sizing configurations:")
batch_results = []
for config in batch_configs:
    strategy = ExpectedValueStrategyCorrected(
        storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
        transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct'],
        min_net_benefit=50,
        **config
    )
    result = engine.run_backtest(strategy)
    batch_results.append(result['net_earnings'])
    print(f"  {config}: ${result['net_earnings']:,.2f}")

unique_batch_results = len(set(batch_results))
if unique_batch_results > 1:
    print(f"\n✓ BATCH SENSITIVITY CONFIRMED: {unique_batch_results} different results")
else:
    print(f"\n⚠️  NO BATCH SENSITIVITY")

## Save Results

In [0]:
import json

results = {
    'timestamp': datetime.now().isoformat(),
    'commodity': COMMODITY,
    'model_version': MODEL_VERSION,
    'baseline': {
        'name': best_baseline_name,
        'earnings': float(best_baseline_earnings)
    },
    'buggy_expected_value': {
        'earnings': float(buggy_ev_earnings),
        'vs_baseline_pct': float(100*(buggy_ev_earnings - best_baseline_earnings)/best_baseline_earnings)
    },
    'fixed_expected_value': {
        'earnings': float(fixed_result['net_earnings']),
        'vs_baseline_pct': float(fixed_vs_baseline),
        'trades': fixed_result['num_trades'],
        'storage_costs': float(fixed_result['total_storage_costs']),
        'final_inventory': float(fixed_result['final_inventory'])
    },
    'corrected_expected_value': {
        'earnings': float(corrected_result['net_earnings']),
        'vs_baseline_pct': float(corrected_vs_baseline),
        'trades': corrected_result['num_trades'],
        'storage_costs': float(corrected_result['total_storage_costs']),
        'final_inventory': float(corrected_result['final_inventory'])
    },
    'parameter_sensitivity': {
        'min_net_benefit': {
            'values': min_benefit_values,
            'earnings': [float(x) for x in min_benefit_results],
            'unique_results': unique_results,
            'range': float(max(min_benefit_results) - min(min_benefit_results))
        },
        'batch_sizing': {
            'earnings': [float(x) for x in batch_results],
            'unique_results': unique_batch_results
        }
    },
    'improvement': {
        'fixed_to_corrected_pct': float(swing_fixed_to_corrected),
        'fixed_to_corrected_dollars': float(corrected_result['net_earnings'] - fixed_result['net_earnings'])
    }
}

# Save to Volume
json_path = '/Volumes/commodity/trading_agent/files/diagnostic_14_results.json'
with open(json_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"✓ Results saved to: {json_path}")

# Also save pickle with full details
pkl_path = '/Volumes/commodity/trading_agent/files/diagnostic_14_results.pkl'
with open(pkl_path, 'wb') as f:
    pickle.dump({
        'results': results,
        'fixed_trades': fixed_result['trades'],
        'corrected_trades': corrected_result['trades']
    }, f)

print(f"✓ Detailed results saved to: {pkl_path}")

print("\n" + "="*80)
print("DIAGNOSTIC 14 COMPLETE")
print("="*80)